In [1]:
#import dependencies 
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
#define folder path, flower name list and ouput dictionary
folder_path = "Data/Kaggle_images/test/"
flower_list = ['daisy','dandelion','rose','sunflower','tulip']
output = {"model":[],"flower_name":[],"predict_accuracy":[]}

In [3]:
#define the pretrained model list, include all models at Keras and their default image sizes respectively
model_list = [
    {'model_name': 'Xception','default_dim':(299, 299)},
    {'model_name': 'VGG16','default_dim':(224, 224)},
    {'model_name': 'VGG19','default_dim':(224, 224)},
    {'model_name': 'ResNet50','default_dim':(224, 224)},
    {'model_name': 'InceptionV3','default_dim':(299, 299)},
    {'model_name': 'InceptionResNetV2','default_dim':(299, 299)},
    {'model_name': 'MobileNet','default_dim':(224, 224)},
    {'model_name': 'DenseNet121','default_dim':(224, 224)},
    {'model_name': 'NASNetLarge','default_dim':(331, 331)},
    {'model_name': 'MobileNetV2','default_dim':(224, 224)}
]

#loop through the model list and capture the prediction accuracy for each model
for model in model_list:
    selected_model = model['model_name']
    image_size = model['default_dim']
    #determine the import module name per model selected
    if selected_model == 'InceptionV3':
        import_module = 'keras.applications.inception_v3'
    elif selected_model == 'InceptionResNetV2':
        import_module = 'keras.applications.inception_resnet_v2'
    elif selected_model == 'DenseNet121':
        import_module = 'keras.applications.densenet'
    elif selected_model == 'NASNetLarge':
        import_module = 'keras.applications.nasnet'
    else:
        import_module = 'keras.applications.' + selected_model.lower()
    
    #import model and functions from the module
    imported_model = getattr(__import__(import_module, fromlist=[selected_model]), selected_model)
    preprocess_input = getattr(__import__(import_module, fromlist=['preprocess_input']), 'preprocess_input')
    decode_predictions = getattr(__import__(import_module, fromlist=['decode_predictions']), 'decode_predictions')
    
    #create my test model with the imported model
    my_model = imported_model(
        include_top=True,
        weights='imagenet')

    #loop through all flower names or classes in the flower list
    for flower in flower_list:
        #get the list of image file names under each flower class
        image_list = os.listdir(os.path.join(folder_path,flower))
        #determine the total image count
        total_count = len(image_list)
        #set up counter for correct predictions and a list for predicted results
        correct_count = 0
        predict_result = []
        #loop through all image files in the list
        for image_file in image_list:
            #if there is a windows Thumbs.db file, skip it and correct the total image count
            if image_file == 'Thumbs.db':
                total_count -= 1
            else:
                #get the path and file name of each image
                image_path = os.path.join(folder_path,flower, image_file)
                #load image and pre-process it
                img = image.load_img(image_path, target_size=image_size)
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
                #predict the flower name with the model
                predictions = my_model.predict(x)
                #get the result of predictions and add to list
                result = decode_predictions(predictions, top=1)[0][0][1]
                predict_result.append(result)
                #if the predicted flower name is same as the flower class name, then correct count increase
                if result.lower() == flower:
                    correct_count += 1
        #update the output dictionary with model name, flower name and prediction accuracy
        output["flower_name"].append(flower)
        output["predict_accuracy"].append(correct_count / total_count)
        output["model"].append(selected_model)


In [4]:
#read output into pandas
df = pd.DataFrame.from_dict(output)
df

,flower_name,model,predict_accuracy
0,daisy,Xception,0.800000
1,dandelion,Xception,0.000000
2,rose,Xception,0.000000
3,sunflower,Xception,0.000000
4,tulip,Xception,0.000000
5,daisy,VGG16,0.760000
6,dandelion,VGG16,0.000000
7,rose,VGG16,0.000000
8,sunflower,VGG16,0.000000
9,tulip,VGG16,0.000000


In [5]:
 #transform data to the right format for analysis
df_formatted = df.pivot(index='model',columns='flower_name')
df_formatted.sort_values([('predict_accuracy','daisy')],ascending = False)

predict_accuracy                               
flower_name                  daisy dandelion rose sunflower tulip
model                                                            
InceptionResNetV2         0.853333       0.0  0.0       0.0   0.0
InceptionV3               0.840000       0.0  0.0       0.0   0.0
NASNetLarge               0.840000       0.0  0.0       0.0   0.0
Xception                  0.800000       0.0  0.0       0.0   0.0
MobileNet                 0.786667       0.0  0.0       0.0   0.0
MobileNetV2               0.773333       0.0  0.0       0.0   0.0
VGG19                     0.773333       0.0  0.0       0.0   0.0
ResNet50                  0.760000       0.0  0.0       0.0   0.0
VGG16                     0.760000       0.0  0.0       0.0   0.0
DenseNet121               0.746667       0.0  0.0       0.0   0.0